# *Анализ базы данных о книгах*

<font face='liberation serif' size=4>
    
Целью исследования является анализ базы данных, содержащей информацию о книгах, издательствах, авторах и пользовательских обзорах книг, с целью сформулировать ценностное предложение для нового продукта - сервиса для чтения книг по подписке.    
    
    
    
**Ответить на вопросы:**
1. Определить количество книг, изданных в XXI веке.

2. Оценить популярность и качество каждой книги в базе данных.

3. Выявить наиболее продуктивное издательство в отношении выпуска книг большого объема.

4. Выявить автора, чьи книги пользователи считают наиболее качественными.

5. Оценить активность пользователей, которые оставляют много обзоров.

Таким образом, исследование будет включать в себя задачи:<br>

- [x] Посчитать, сколько книг вышло после 1 января 2000 года;<br>
- [x] Для каждой книги посчитать количество обзоров и среднюю оценку;<br>
- [x] Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;<br>
- [x] Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;<br>
- [x] Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.<br>

    
**Исходные данные:**
  
***Таблица books***<br>
Содержит данные о книгах:<br>
book_id — идентификатор книги;<br>
author_id — идентификатор автора;<br>
title — название книги;<br>
num_pages — количество страниц;<br>
publication_date — дата публикации книги;<br>
publisher_id — идентификатор издателя.<br>

***Таблица authors***<br>
Содержит данные об авторах:<br>
author_id — идентификатор автора;<br>
author — имя автора.<br>

***Таблица publishers***<br>
Содержит данные об издательствах:<br>
publisher_id — идентификатор издательства;<br>
publisher — название издательства;<br>

***Таблица ratings***<br>
Содержит данные о пользовательских оценках книг:<br>
rating_id — идентификатор оценки;<br>
book_id — идентификатор книги;<br>
username — имя пользователя, оставившего оценку;<br>
rating — оценка книги.<br>

***Таблица reviews***<br>
Содержит данные о пользовательских обзорах:<br>
review_id — идентификатор обзора;<br>
book_id — идентификатор книги;<br>
username — имя автора обзора;<br>
text — текст обзора.<br>    


In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
pd.set_option('max_colwidth', 200)

In [3]:
db_config = {'user': 'praktikum_student', 
'pwd': 'Sdf4$2;d-d30pp', 
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, 
'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()

In [4]:
def query(query):
    display(pd.io.sql.read_sql(sql=text(query), con = con))

In [5]:
for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    print('Таблица —', table)
    query(
    f'''
    SELECT * 
    FROM {table} 
    LIMIT 5
    '''
     )
    print('Кол-во строк:')
    query(
    f'''
    SELECT COUNT(*) 
    FROM {table} 
'''
    )
    print()

Таблица — books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Кол-во строк:


,count
0,1000



Таблица — authors


...

,count
0,6456



Таблица — reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


Кол-во строк:


,count
0,2793


In [6]:
print('Минимальное и максимальное количество количество страниц, \nа также самый ранний и самый поздний годы публикации')
query('''
SELECT
    MIN(num_pages) AS min_pages,
    MAX(num_pages) AS max_pages,
    CAST(MIN(EXTRACT(YEAR FROM publication_date)) as int)  AS min_publication_year,
    CAST(MAX(EXTRACT(YEAR FROM publication_date)) as int) AS max_publication_year
FROM
    books
''')

Минимальное и максимальное количество количество страниц, 
а также самый ранний и самый поздний годы публикации


,min_pages,max_pages,min_publication_year,max_publication_year
0,14,2690,1952,2020


In [7]:
print('Минимальные и максимальные рейтинги')
query('''
SELECT
    MIN(rating) AS min_rating,
    MAX(rating) AS max_rating
FROM
    ratings
''')

Минимальные и максимальные рейтинги


,min_rating,max_rating
0,1,5


<font face='liberation serif' size=4>
    
    
    
В таблице books содержится информация о 1000 книгах, количество страниц которых варьируется от 14 до 2690. Самая ранняя книга была опубликована в 1952 году, а самая поздняя — в 2020 году.<br>
В таблице authors есть данные о 636 авторах.<br>
В таблице publishers представлена информация о 340 издательствах.<br>
Таблица ratings включает 6456 записей с оценками книг по пятибалльной шкале, которые выставили пользователи. <br>
Также есть таблица с 2793 записями о пользовательских обзорах книг.

In [8]:
print('Кол-во книг после 1 января 2000 года:')
query('''
SELECT COUNT(*)
FROM books
WHERE publication_date > '2000-01-01'
''')

Кол-во книг после 1 января 2000 года:


,count
0,819


<font face='liberation serif' size=4>
    
<b>Количество книг, опубликованных после 1 января 2000 года, равно 819.

In [9]:
print('Количество обзоров и средняя оценка для каждой книги:')
query('''
SELECT 
    b.book_id,
    b.title,
    COUNT(DISTINCT r.review_id) AS review_count,
    AVG(rt.rating) AS average_rating
FROM books b
LEFT JOIN reviews r ON b.book_id = r.book_id
LEFT JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title
ORDER BY 
    average_rating DESC,
    review_count DESC
''')



Количество обзоров и средняя оценка для каждой книги:


,book_id,title,review_count,average_rating
0,17,A Dirty Job (Grim Reaper #1),4,5.00
1,553,School's Out—Forever (Maximum Ride #2),3,5.00
2,444,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
4,972,Wherever You Go There You Are: Mindfulness Meditation in Everyday Life,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twenty-first Century,3,2.25
996,202,Drowning Ruth,3,2.00
997,316,His Excellency: George Washington,2,2.00
998,371,Junky,2,2.00


<font face='liberation serif' size=4>
<b>Максимальное количество отзывов на одну книгу – 4, а минимальное – 2. Средняя максимальная оценка – 5, <br>а минимальная – 1,5.

In [10]:
print('Издательство, которое выпустило наибольшее число книг толще 50 страниц:')
query('''
SELECT 
    p.publisher,
    COUNT(b.book_id) AS book_count
FROM 
    publishers p
JOIN 
    books b ON p.publisher_id = b.publisher_id
WHERE 
    b.num_pages > 50
GROUP BY 
    p.publisher
ORDER BY book_count DESC
LIMIT 1
''')

Издательство, которое выпустило наибольшее число книг толще 50 страниц:


,publisher,book_count
0,Penguin Books,42


<font face='liberation serif' size=4>
    
<b>Издательство "Penguin Books" выпустило 42 книги, объём которых превышает 50 страниц. Это рекордное количество среди всех издательств.

In [11]:
print('Автор с самой высокой средней оценкой книг — с 50 и более оценками:')
query('''
SELECT 
    a.author_id,
    a.author,
    AVG(r.rating) AS average_rating,
    COUNT(r.rating) AS Count_r
FROM 
    authors AS a
JOIN 
    books AS b ON a.author_id = b.author_id
JOIN 
    ratings AS r ON b.book_id = r.book_id
GROUP BY 
     a.author_id, a.author, r.rating
HAVING 
    COUNT(r.rating) >= 50
ORDER BY 
    count_r DESC
LIMIT 1
''')

Автор с самой высокой средней оценкой книг — с 50 и более оценками:


,author_id,author,average_rating,count_r
0,236,J.K. Rowling/Mary GrandPré,5.0,160


<font face='liberation serif' size=4>
<b>Автор Diana Gabaldon имеет рейтинг 4,3 и это самый высокий средний показатель среди авторов. У неё более 50 оценок.

In [12]:
print('Среднее количество обзоров от пользователей, которые поставили больше 48 оценок:')
query('''
WITH user_ratings AS (
    SELECT
        username,
        COUNT(rating_id) AS rating_count
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 48
),
user_reviews AS (
    SELECT
        username,
        COUNT(review_id) AS review_count
    FROM reviews
    GROUP BY username
)

SELECT
    AVG(rev.review_count) AS average_reviews
FROM
    user_ratings AS ur
JOIN
    user_reviews AS rev ON ur.username = rev.username
''')


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок:


,average_reviews
0,24.0


<font face='liberation serif' size=4>
<b>Среднее количество отзывов от пользователей, которые оценили книги больше 48 раз, составляет 24

<font face='liberation serif' size=4>

# Вывод:


- Информация о 1000 книгах охватывает широкий диапазон страниц (от 14 до 2690), с публикациями, датируемыми от 1952 до 2020 годов.
- В данных представлено 636 авторов и 340 издательств, что указывает на разнообразие контента и источников.
- Отзывы пользователей включают 6456 оценок, что свидетельствует о активном взаимодействии читателей с книгами, а также о наличии 2793 пользовательских обзоров.
- Значительное количество книг — 819 — было опубликовано после 2000 года.
- Оценки книг по пятибалльной шкале варьируются от 1,5 до 5, а максимальное количество отзывов на книгу составляет 4.
- Издательство "Penguin Books" установило рекорд, выпустив 42 книги объемом более 50 страниц.
- Автор Diana Gabaldon выделяется на фоне других, имея наивысший средний рейтинг в 4,3 при более чем 50 оценках.
- Среднее количество отзывов для книг, оценённых более чем 48 раз, составляет 24, что подчеркивает заинтересованность пользователей в более популярных произведениях.